In [1]:
include("TDCM_full_transition.jl")
include("data_generation.jl")
using Distributions

In [2]:
N, J, K, O, L, S = 5000, 24, 3, 3, 8, 50
num_features_gamma = 3
Q = [0 0 0; 0 0 1; 0 1 0; 1 0 0; 0 1 1; 1 0 1; 1 1 0; 1 1 1;
     0 0 0; 0 0 1; 0 1 0; 1 0 0; 0 1 1; 1 0 1; 1 1 0; 1 1 1;
     0 0 0; 0 0 1; 0 1 0; 1 0 0; 0 1 1; 1 0 1; 1 1 0; 1 1 1]

X = Vector{Vector{Matrix{Float64}}}(undef, K)
for k in 1:K
     X[k] = Vector{Matrix{Float64}}(undef, O)
     X[k][1] = ones(N, 1)
     for t in 2:O
          X[k][t] = Matrix{Float64}(undef, N, 3)
          X[k][t][:,1] = ones(N)
          X[k][t][:,2] = rand(Normal(0, 1), N)
          X[k][t][:,3] = rand(Bernoulli(0.5), N)
     end
end
U = Vector{Vector{Matrix{Float64}}}(undef, K)
for k in 1:K
     U[k] = Vector{Matrix{Float64}}(undef, O)
     for t in 1:O
          U[k][t] = Matrix{Float64}(undef, S, 3)
          U[k][t][:,1] = ones(S)
          U[k][t][:,2] = rand(Bernoulli(0.5), S)
          U[k][t][:,3] = rand(Bernoulli(0.5), S)
     end
end

In [3]:
data = data_generation(Q, X, U, Int(O), Int(K), Int(S), Int(N/S), beta_interact=true);

In [4]:
obs = TDCMObs(data.Y, data.Q, data.U, data.X, data.group);

In [5]:
N, O, J, K, L, S = size(obs.Y, 1), size(obs.Y, 2), size(obs.Y, 3),  size(obs.Q, 2), size(obs.D[1], 1), size(obs.U[1][1], 1)

mu_beta_prior = Vector{Vector{Float64}}(undef, J)
L_beta_prior = Vector{Matrix{Float64}}(undef, J)
for j in 1:J
    num_features = size(obs.D[j], 2)
    mu_beta_prior[j] = zeros(num_features)
    L_beta_prior[j] = Matrix(1.0I, num_features, num_features)
end

mu_omega_prior = Vector{Vector{Vector{Vector{Vector{Float64}}}}}(undef, K)
L_omega_prior = Vector{Vector{Vector{Vector{Matrix{Float64}}}}}(undef, K)
a_tau_prior = Vector{Vector{Vector{Vector{Float64}}}}(undef, K)
b_tau_prior = Vector{Vector{Vector{Vector{Float64}}}}(undef, K)

for k in 1:K
    mu_omega_prior[k] = Vector{Vector{Vector{Vector{Float64}}}}(undef, O)
    L_omega_prior[k] = Vector{Vector{Vector{Matrix{Float64}}}}(undef, O)
    a_tau_prior[k] = Vector{Vector{Vector{Float64}}}(undef, O)
    b_tau_prior[k] = Vector{Vector{Vector{Float64}}}(undef, O)
    for t in 1:O
        num_features_gamma = size(obs.X[k][t], 2)
        num_features_omega = size(obs.U[k][t], 2)
        if t == 1
            mu_omega_prior[k][t] = Vector{Vector{Vector{Float64}}}(undef, 1)
            mu_omega_prior[k][t][1] = Vector{Vector{Float64}}(undef, 1)
            mu_omega_prior[k][t][1][1] = zeros(num_features_omega)

            L_omega_prior[k][t] = Vector{Vector{Matrix{Float64}}}(undef, 1)
            L_omega_prior[k][t][1] = Vector{Matrix{Float64}}(undef, 1)
            L_omega_prior[k][t][1][1] = Matrix{Float64}(1.0I, num_features_omega, num_features_omega)

            a_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 1)
            a_tau_prior[k][t][1] = [.01]

            b_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 1)
            b_tau_prior[k][t][1] = [.01]
        else
            mu_omega_prior[k][t] = Vector{Vector{Vector{Float64}}}(undef, 2)
            L_omega_prior[k][t] = Vector{Vector{Matrix{Float64}}}(undef, 2)
            a_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 2)
            b_tau_prior[k][t] = Vector{Vector{Float64}}(undef, 2)
            for z in 1:2
                mu_omega_prior[k][t][z] = Vector{Vector{Float64}}(undef, num_features_gamma)
                L_omega_prior[k][t][z] = Vector{Matrix{Float64}}(undef, num_features_gamma)
                a_tau_prior[k][t][z] = ones(num_features_gamma) .* .01
                b_tau_prior[k][t][z] = ones(num_features_gamma) .* .01
                for m in 1:num_features_gamma
                    mu_omega_prior[k][t][z][m] = zeros(num_features_omega)
                    L_omega_prior[k][t][z][m] = Matrix{Float64}(1.0I, num_features_omega, num_features_omega)
                end
            end
        end
    end
end

In [14]:
M = 10
model = TDCModel(obs, mu_beta_prior, L_beta_prior, mu_omega_prior, L_omega_prior, a_tau_prior, b_tau_prior, M, enable_parallel=true)
optim = ADAMparams(model);

TDCModel constructed for computation on 12 threads


In [15]:
#Initialize betas in the correct direction
for j in 1:J
    model.mu_beta_star[j] = ones(length(model.mu_beta_star[j]))
    model.mu_beta_star[j][1] = -1
end

# Sample Z values
for i in 1:N
    for t in 1:O
        sample_Z(model, i, t)
    end
end

# Sample beta
sample_β(model)

# Sample gammas
for k in 1:K
    for t in 1:O
        if t == 1
            for s in 1:S
                sample_γ(model, s, t, k, 1)
            end
        else
            for z in 1:2
                for s in 1:S
                    sample_γ(model, s, t, k, z)
                end
            end
        end
    end
end

# Sample omega and tau values
for k in 1:K
    for t in 1:O
        num_features = length(model.gamma_sample[k][t][1][1][1])
        if t == 1
            for feature in 1:num_features
                sample_ω(model, k, t, 1, feature)
                sample_τ(model, k, t, 1, feature)
            end
        else
            for z in 1:2
                for feature in 1:num_features
                    sample_ω(model, k, t, z, feature)
                    sample_τ(model, k, t, z, feature)
                end
            end
        end
    end
end

In [16]:
for iter in 1:300
    update_categorical_variational_distribution(model, 1, step=.001, maxiter=5, verbose=false)
    update_categorical_variational_distribution(model, 2, step=.001, maxiter=5, verbose=false)
    update_normal_variational_distribution(model, init_step = .0001, maxiter = 5, clip = 50.0, verbose = false, ADAMparams=optim)
    update_normal_variational_distribution2(model, init_step = .001, maxiter = 10, clip = 50.0, verbose = false, ADAMparams=optim)
    update_normal_variational_distribution3(model, init_step = .001, maxiter = 10, clip = 50.0, verbose = false, ADAMparams=optim)
    update_inverse_gamma_distribution(model, step = .001, maxiter = 5, verbose = false, ADAMparams=optim)
end

1, 3, 4, 14
1, 3, 3, 7
1, 3, 3, 36
1, 3, 4, 43
1, 3, 3, 29
1, 3, 3, 22
1, 3, 3, 1


LoadError: TaskFailedException

[91m    nested task error: [39mPosDefException: matrix is not Hermitian; Cholesky factorization failed.
    Stacktrace:
      [1] [0m[1mcheckpositivedefinite[22m[0m[1m([22m[90minfo[39m::[0mInt64[0m[1m)[22m
    [90m    @[39m [36mLinearAlgebra[39m [90mC:\Users\14842\AppData\Local\Programs\Julia-1.10.2\share\julia\stdlib\v1.10\LinearAlgebra\src\[39m[90m[4mfactorization.jl:67[24m[39m
      [2] [0m[1mcholesky![22m[0m[1m([22m[90mA[39m::[0mMatrix[90m{Float64}[39m, ::[0mNoPivot; [90mcheck[39m::[0mBool[0m[1m)[22m
    [90m    @[39m [36mLinearAlgebra[39m [90mC:\Users\14842\AppData\Local\Programs\Julia-1.10.2\share\julia\stdlib\v1.10\LinearAlgebra\src\[39m[90m[4mcholesky.jl:298[24m[39m
      [3] [0m[1mcholesky![22m[90m (repeats 2 times)[39m
    [90m    @[39m [90mC:\Users\14842\AppData\Local\Programs\Julia-1.10.2\share\julia\stdlib\v1.10\LinearAlgebra\src\[39m[90m[4mcholesky.jl:295[24m[39m[90m [inlined

In [17]:
model.V_gamma_star[1][3][3][1]

3×3 Matrix{Float64}:
  Inf  NaN  NaN
 NaN   NaN  NaN
 NaN   NaN  NaN

In [18]:
model.storage_grad_L_par

12-element Vector{Vector{Float64}}:
 [-Inf, Inf, Inf, 12868.950663894377, -27962.0233418303, 9869.444969369542, -25414.19504609596, -5821.390520774465]
 [-50.18258567974776, 3977.410382441626, 3.2915703561801193, 3.297912917961873, 3.2925330207686487, 3.3010415670652833, 3.3036539746136873, 3.309976563813477]
 [Inf, Inf, -Inf, 1.775513495061018, 1.7752949161954104, 1.773112671168238, 1.7696389626129272, 1.7578007990451512]
 [-367.5203617018443, -439.40880495526534, -270.1637614283896, -22.13325023180693, 3.163090441854432, 3.1657964354179, 3.1644108572652714, 3.160911576295537]
 [-154.09765692539239, -312.3667886509705, 134.81830066939264, 2.6167499775588987, 2.620424939460397, 2.6114120468772173, 2.615747751690629, 2.6128134544613713]
 [Inf, Inf, Inf, -6496.378157316934, 2.7671886863636845, -19.322877536870052, 2.76558937237041, 2.7542448047833754]
 [-51.44147230744061, -36.3317942121499, 53.18314953689884, 2564.687250090563, 6009.342622964908, -1847.6851090977002, 15739.21101710165, 

In [19]:
model.storage_LL2_par

12-element Vector{Matrix{Float64}}:
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.5998733307380518 0.16902033981824535 … -0.3783472097277249 0.0; 0.43988225743677156 0.27893996123345094 … 0.4366615141583316 -0.22989979903007576]
 [-0.6763443151373586 0.0 … 6.9527700721797e-310 6.9527700721797e-310; -0.7910825917955041 -0.6890908297222517 … 6.9527700724105e-310 6.9527700724105e-310; … ; 6.9527700721797e-310 6.9527700721797e-310 … 6.9527700721797e-310 6.9527700724105e-310; 6.9527700724105e-310 6.9527700724105e-310 … 6.9527700724105e-310 0.0]
 [0.0 0.0 … 6.9527700721797e-310 6.9527700721797e-310; 0.0 0.0 … 6.9527700724105e-310 6.9527700724105e-310; … ; 6.9527700721797e-310 6.9527700721797e-310 … 6.9527700721797e-310 6.9527700724105e-310; 6.9527700724105e-310 6.9527700724105e-310 … 6.9527700724105e-310 6.95276652756143e-310]
 [-0.7865503724686799 0.0 … 1.4247881989206e-311 1.4247881990787e-311; 0.1978739784979701 -0.901058654001444 … 1.4247881989364e-311 1.4247881990945e-311; … ; 1.4247881

In [13]:
data.mu_gamma_star[1][3][3][1]

3-element Vector{Float64}:
  3.063339155488862
  1.2108126901587353
 -1.4172682596734025